# Laws textures 3D

In [1]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

## Load the train set

In [2]:
data_flag = 'NoduleMNIST3D'.lower()
download = True
BATCH_SIZE = 4

info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])

# load the data
train_dataset = DataClass(split='train',  download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle = True)

Using downloaded and verified file: /Users/gravo/.medmnist/nodulemnist3d.npz


### Compute the train set features

In [3]:
from feature_maps import laws_textures

trainset = []
laws = laws_textures()

for batch in tqdm(train_loader):
    for i in range(len(batch[1])):
        sample = {}
        sample["image"] = batch[0][i].squeeze()
        sample["label"] = batch[1][i]
        sample["laws"] = laws.get_features(sample["image"])
        trainset.append(sample)

100%|██████████| 290/290 [09:46<00:00,  2.02s/it]


In [4]:
for sample in trainset:
    features = []
    for i in range(sample["laws"].shape[-1]):
        features.append( sample["laws"][...,i].mean() )
        features.append( sample["laws"][...,i].std() )
    sample["features"] = features

In [5]:
X = [sample["features"] for sample in trainset]
Y = [sample["label"].numpy()[0] for sample in trainset]

_, c = np.unique(Y, return_counts = True)
print("Data distribution", c)

Data distribution [863 295]


## Train the classifier

In [6]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter = 10000)
lr.fit(X, Y)

LogisticRegression(max_iter=10000)

In [7]:
import sklearn.metrics as metrics

def compute_metrics(y_true, y_prob):
    y_pred      = (y_prob > .5).astype(int)
    accuracy    = metrics.accuracy_score    (y_true = y_true, y_pred = y_pred)
    precision   = metrics.precision_score   (y_true = y_true, y_pred = y_pred)
    recall      = metrics.recall_score      (y_true = y_true, y_pred = y_pred)
    f1_score    = metrics.f1_score          (y_true = y_true, y_pred = y_pred)
    auc         = metrics.roc_auc_score     (y_true = y_true, y_score = y_prob)
    return {"accuracy": accuracy, "precision": precision, 
            "recall": recall, "f1-score": f1_score, "auc": auc}

prob = lr.predict_proba(X)
compute_metrics(Y, prob[:,1])

{'accuracy': 0.8281519861830743,
 'precision': 0.7891566265060241,
 'recall': 0.4440677966101695,
 'f1-score': 0.5683297180043385,
 'auc': 0.7887267513796963}

## Test the classifier

In [8]:
data_flag = 'NoduleMNIST3D'.lower()
download = True
BATCH_SIZE = 4

info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])

# load the data
test_dataset = DataClass(split='test',  download=download)

# encapsulate data into dataloader form
test_loader = data.DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE, shuffle = True)

Using downloaded and verified file: /Users/gravo/.medmnist/nodulemnist3d.npz


In [9]:
testset = []
laws = laws_textures()

for batch in tqdm(test_loader):
    for i in range(len(batch[1])):
        sample = {}
        sample["image"] = batch[0][i].squeeze()
        sample["label"] = batch[1][i]
        sample["laws"] = laws.get_features(sample["image"])
        testset.append(sample)

100%|██████████| 78/78 [02:47<00:00,  2.15s/it]


In [10]:
for sample in testset:
    features = []
    for i in range(sample["laws"].shape[-1]):
        features.append( sample["laws"][...,i].mean() )
        features.append( sample["laws"][...,i].std() )
    sample["features"] = features

In [11]:
X_test = [sample["features"] for sample in testset]
Y_test = [sample["label"].numpy()[0] for sample in testset]

_, c = np.unique(Y_test, return_counts = True)
print("Data distribution", c)

Data distribution [246  64]


In [12]:
prob_test = lr.predict_proba(X_test)
compute_metrics(Y_test, prob_test[:,1])

{'accuracy': 0.832258064516129,
 'precision': 0.65,
 'recall': 0.40625,
 'f1-score': 0.5000000000000001,
 'auc': 0.7231326219512195}